In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import json
import os

# Define the base URL and list of generations to scrape
base_url = 'https://www.wikidex.net/wiki/'
""" gens = ['Lista_de_Pokémon_de_la_primera_generación',
        'Lista_de_Pokémon_de_la_segunda_generación',
        'Lista_de_Pokémon_de_la_tercera_generación',
        'Lista_de_Pokémon_de_la_cuarta_generación',
        'Lista_de_Pokémon_de_la_quinta_generación',
        'Lista_de_Pokémon_de_la_sexta_generación'] """

gens = ['Lista_de_Pokémon_de_la_tercera_generación']
pokedex_url = './dexes/'

# Set up the web driver options and service
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
service = Service(ChromeDriverManager().install())

In [ ]:
import random
import string
from dataclasses import dataclass, field, asdict

def generate_id() -> str:
    return "".join(random.choices(string.ascii_uppercase, k=10))

@dataclass(slots=True)
class Pokemon:
    id: str = field(default_factory=generate_id, init=False)
    numDex: str
    species: str
    type1: str
    type2: str
    sprite: str
    weight: float
    abilities: list
    level: int = 50
    ivs: list[int] = field(default_factory=list)
    evs: list[int] = field(default_factory=list)
    baseStats: list[int] = field(default_factory=list)
    item: str = 'Abomasite'
    nature: str = 'Adamant'
    moves: list = field(default_factory= lambda: ['Protect','','',''])
    
def load_from_file(file_url):
    with open(file_url, 'r', encoding="UTF-8") as fp:
        data = json.load(fp)
        
    return data

translations = load_from_file('i18n_eng.json')

def i18n(phrase):
    return translations[phrase]

In [ ]:
def scrape_pokemon(driver, pkmn_name):
    # Connect to the website for the current Pokemon
    driver.get(base_url + pkmn_name)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract the Pokemon type information from the HTML
    pkmn_weight = soup.find("tr", attrs={"title": "Peso del Pokémon"}).td.text
    pkmn_weight = pkmn_weight.replace("kg", "").replace(",", ".").strip()

    # Find all the abilities of a Pokemon
    pkmn_abilities = soup.find("tr", attrs={"title": "Habilidades que puede conocer"}).td.findAll("a")
    """ find_attr(soup, 'tr', 'title','Habilidades que puede conocer') """
    
    
    # Extract the Pokemon ability information from the HTML
    pkmn_abilities = soup.select('tr[title^="Habilidades que puede conocer"] td a')
    ha_2 = soup.select_one('tr[title="Habilidad oculta"] td a').text if soup.select_one('tr[title="Habilidad oculta"]') else None
    num_abilities = len(pkmn_abilities) + (1 if ha_2 else 0)
    abilities = {idx: i18n(ab.text) for idx, ab in enumerate(pkmn_abilities) if len(ab.text) > 1}
    if ha_2:
        abilities['ha'] = i18n(ha_2)
    
    
    # Locate the table of stats (below an h2 with a specific id)
    pkmn_table_stats = soup.find("span", attrs={"id": "Caracter.C3.ADsticas_de_combate"}).parent.find_next('table')

    stats = []
    stat_rows = pkmn_table_stats.find_all("tr")


    # Extract the stats from the table
    for idx in range(1, 7):
        stat_value = int(stat_rows[idx].find_all("td")[0].text.strip())
        stats.append( stat_value )

    # Return a dictionary with the Pokemon information
    return {'weight': float(pkmn_weight), 'abilities': abilities, 'baseStats':stats}

In [ ]:
import logging
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define a function to save the list of Pokemon instances to a JSON file
def save_file(pokedex_url, gen, pkmn_list):
    with open(pokedex_url + gen + '.json', 'w', encoding="UTF-8") as fp:
        json.dump(pkmn_list, fp, indent=4, ensure_ascii=False)

# Define a function to search for the sprite of a Pokemon
def searchSprite(driver, soup, num_dex):
    numero = int(num_dex)
    return 'https://cdn.toast-studio.com/typedex/home/_' + str(numero) + '_regular.png'

# Set up the web driver and scrape data for each generation and Pokemon
start_time = time.time()
with webdriver.Chrome(service=service, options=options) as driver:
    for gen in gens:
        numDexTemp = 0
        try:
            driver.get(base_url + gen)
            pkmn_soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Removing rare things
            for s in pkmn_soup.select('sup'):
                s.extract()
                
            results = pkmn_soup.select('table.tabpokemon tbody tr')
            pkmn_list = []
            for result in results:
                fields = result.select('td')
                
                numDexRow = fields[0]
                if(numDexRow.has_attr('rowspan')):
                    numDexTemp = numDexRow.text.strip()
                
                if len(fields) == 4:
                    numDexRow = numDexTemp
                    pkmn_dex_num = numDexTemp
                    imageRow = fields[0]
                    nameRow = fields[1]
                    typesRow = fields[2]
                elif len(fields) == 3:
                    numDexRow = numDexTemp
                    pkmn_dex_num = numDexTemp
                    imageRow = fields[0]
                    nameRow = fields[1]
                elif len(fields) == 2:
                    numDexRow = numDexTemp
                    pkmn_dex_num = numDexTemp
                    imageRow = fields[0]
                    typesRow = fields[1]
                else:
                    imageRow = fields[1]
                    nameRow = fields[2]
                    typesRow = fields[3]
                    pkmn_dex_num = numDexRow.text.strip()
                
                pkmn_name = nameRow.select_one('a').get('title')
                logging.info(f'{pkmn_dex_num}. {pkmn_name} ({gen})')
                
                pkmn_types = [a.get('title').replace('Tipo ', '') for a in typesRow.select('a')]
                tipo1 = pkmn_types[0]
                tipo2 = pkmn_types[1] if len(pkmn_types) > 1 else 'undefined'
                
                pkmn_sprite = searchSprite(driver, pkmn_soup, pkmn_dex_num)
                
                # Scrape the Pokemon data
                pkmn_data = scrape_pokemon(driver, pkmn_name)
                
                # Create a new Pokemon instance with the scraped data
                pkmn = Pokemon(
                    numDex= pkmn_dex_num,
                    species= pkmn_name,
                    type1= i18n(tipo1),
                    type2= i18n(tipo2),
                    sprite= pkmn_sprite,
                    weight= pkmn_data['weight'],
                    abilities= pkmn_data['abilities'],
                    level=50,
                    ivs=[0, 0, 0, 0, 0, 0],
                    evs=[0, 0, 0, 0, 0, 0],
                    baseStats=pkmn_data['baseStats']
                )
                
                # Add the Pokemon instance to the pkmn_list
                pkmn_list.append(asdict(pkmn))
                
                save_file(pokedex_url, gen, pkmn_list)
            logging.info(f'Successfully saved data for {gen}.')
        except Exception as err:
            logging.error(f'Error: {err}')
            
# Print the time elapsed and a message indicating that the script has finished
elapsed_time = time.time() - start_time
logging.info(f'Script finished in {elapsed_time:.2f} seconds.')
print('Script finished.')